In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [34]:
# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self, device, kernel_size=3, max_pool_size=2, lr=0.001, epochs=3):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=kernel_size, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=max_pool_size, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=kernel_size, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=max_pool_size, stride=2)
        self.fc1 = nn.Linear(32 * 75 * 75, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 4)

        self.lr = lr
        self.epochs = epochs
        self.device = device


    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x
    
    def train_cnn(self, train_dataset, train_labels):
        # Set the loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), self.lr)

        # Create a data loader for the training dataset
        train_loader = DataLoader(dataset=list(zip(train_dataset, train_labels)), batch_size=16, shuffle=True)

        self.train()
        for epoch in range(self.epochs):
            running_loss = 0.0
            for images, labels in train_loader:
                images = images.to(self.device)
                labels = labels.to(self.device)

                optimizer.zero_grad()

                outputs = self(images)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

            running_loss += loss.item() * images.size(0)

            epoch_loss = running_loss / len(train_dataset)
            print(f"Epoch [{epoch+1}/{self.epochs}], Loss: {epoch_loss:.4f}")

    def predict(self, test_dataset):
        predictions = []
        self.eval()  # Set the model to evaluation mode

        # Create a data loader for the training dataset
        dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

        
        with torch.no_grad():  # Disable gradient computation for efficiency
            for inputs in dataloader:
                #inputs = inputs.to(device)  # Move inputs to the same device as the model
                
                # Forward pass through the model to obtain predictions
                outputs = self(inputs)
                
                # Get the predicted class indices
                _, predicted = torch.max(outputs, 1)
                
                # Append the predicted class indices to the predictions list
                predictions.extend(predicted.tolist())
        
        return predictions





In [36]:
# Create an instance of the CNN model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN(epochs=3, device=device).to(device)

# Create a random training dataset for demonstration
train_dataset = torch.randn(100, 1, 300, 300)  # Assuming grayscale images with size 300x300
train_labels = torch.randint(0, 4, (100,))  # Random labels (0 to 3) for the categories

model.train_cnn(train_dataset, train_labels)

# Create a random test dataset for demonstration
test_dataset = torch.randn(10, 1, 300, 300)  # Assuming grayscale images with size 300x300


model.predict(test_dataset)

Epoch [1/3], Loss: 0.2680
Epoch [2/3], Loss: 0.0517
Epoch [3/3], Loss: 0.0544


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]